In [1]:
import numpy as np
from collections import defaultdict

In [2]:
train_datasets = ['gutenberg']
test_datasets  = ['gutenberg']
prefix = '_'.join(train_datasets) + '-' + '_'.join(test_datasets)

word_to_id = np.load('%s_word_to_id.npy' % prefix).item()
id_to_word = np.load('%s_id_to_word.npy' % prefix).item()
n_grams = np.load('%s_n_grams.npy' % prefix).item()
train_sentences = np.load('%s_train.npy' % prefix)
valid_sentences = np.load('%s_valid.npy' % prefix)

In [3]:
# Store conditional counts
conditional_n_grams = dict()

for n in range(1,2+1):
    conditional_n_grams[n] = defaultdict(dict)
    
    for n_gram, count in n_grams[n].iteritems():
        conditional_n_grams[n][n_gram[:-1]][n_gram[-1]] = count

In [4]:
# Store continuation counts
continuations = dict()

for n in range(1,2+1):
    continuations[n] = defaultdict(set)
    
    for n_gram, count in n_grams[n].iteritems():        
        if count > 0:
            continuations[n][n_gram[1:]].add(n_gram[0])

In [5]:
continuation_prob = np.zeros((len(word_to_id) + 1,))
denominator = sum([len(x) for x in continuations[n].values()])

for i in range(len(word_to_id) + 1):
    continuation_prob[i] = float(len(continuations[2][(i,)])) / denominator

In [6]:
def get_distribution(context, d=0.75):
    n = len(context) + 1
    assert n == 2
    
    # create probability distribution by copying continuation probability
    counts = np.array(continuation_prob)
    
    # premultiply with lambda_
    lambda_ = float(d)/n_grams[n-1][context] * len(conditional_n_grams[n][context])
    counts *= lambda_
    
    for i in conditional_n_grams[n][context]:
        bigram_score = float(max(conditional_n_grams[n][context][i] - d, 0))/n_grams[n-1][context]
        counts[i] += bigram_score
        
    return counts

In [7]:
def get_probability(context, next_word):
    probabilities = get_distribution(context)
    return probabilities[next_word]

In [8]:
def compute_perplexity(sentence):
    running_prob = 0

    for pos in range(1, len(sentence)):
        query_n = 1
        if query_n == 0:
            context = tuple()
        else:
            context = tuple(map(lambda x: word_to_id[x] if x in word_to_id else 0, sentence[:pos][-1 * query_n:]))
        next_word = word_to_id[sentence[pos]] if sentence[pos] in word_to_id else 0

        running_prob += np.log(get_probability(context, next_word))
        #print running_prob

    return np.exp(-running_prob/len(sentence))

In [9]:
len(valid_sentences)

9856

In [10]:
ppls = []

for i, sentence in enumerate(valid_sentences):
    temp = compute_perplexity(sentence)
    ppls += [temp]
    print i, temp, np.mean(ppls)

0 292.402342273 292.402342273
1 32.6345676956 162.518454984
2 637.840716286 320.959208752
3 246.764771599 302.410599463
4 287.84997607 299.498474785
5 167.314997541 277.467895244
6 75.369049795 248.596631609
7 352.858405043 261.629353288
8 282.914301194 263.9943475
9 138.564060229 251.451318773
10 66.9508896443 234.678552488
11 266.962166777 237.368853679
12 81.7610070796 225.399019325
13 189.666674624 222.846708989
14 787.32748565 260.478760767
15 30.3019807325 246.092712015
16 171.121347271 241.682631736
17 27.7237870333 229.796029252
18 690.42664506 254.039745874
19 112.460433823 246.960780271
20 615.785121749 264.523844151
21 136.338266851 258.697227001
22 149.545555386 253.951502148
23 131.182996126 248.836147731
24 107.617530354 243.187403035
25 78.2236344927 236.842642707
26 387.102014879 242.407804639
27 412.2809526 248.474702781
28 198.473781531 246.750533082
29 124.347057713 242.670417237
30 214.136330047 241.749962811
31 131.622672164 238.308484979
32 200.324882145 237.15746

262 61.8757199083 188.616929432
263 202.533077993 188.669642116
264 176.376571522 188.623253171
265 7.20113707157 187.94121514
266 103.242816316 187.623992673
267 218.478795824 187.739122535
268 138.13571911 187.554723266
269 139.357857928 187.376216357
270 265.791569873 187.665571905
271 217.782192427 187.776294775
272 165.179349587 187.693522082
273 134.388606258 187.498978594
274 321.291341998 187.985496279
275 60.5730052328 187.523856818
276 131.017172634 187.319861569
277 39.5846620433 186.788439988
278 55.5507903407 186.318054147
279 160.206174255 186.224797433
280 142.535830431 186.069320682
281 51.1006769697 185.59070847
282 182.570093065 185.580034917
283 118.917472668 185.345307586
284 105.351110267 185.064626191
285 87.6034421948 184.723852821
286 65.6750129767 184.309048501
287 211.854923355 184.4046939
288 40.4091522484 183.90643943
289 145.197088309 183.772958909
290 99.5810706833 183.483639706
291 39.6121103884 182.990928989
292 72.9523966009 182.615370858
293 120.695369

527 134.631869042 174.394230107
528 240.106526164 174.518449948
529 66.8947954458 174.315386449
530 176.841787409 174.320144266
531 80.7857298952 174.144327698
532 134.428779102 174.069814474
533 235.943166037 174.185682173
534 248.637460473 174.324844376
535 127.581556926 174.23763675
536 100.801939616 174.100884986
537 163.829081239 174.081792414
538 221.089914097 174.169005998
539 111.753543515 174.053421808
540 226.994918703 174.151280398
541 592.802933321 174.923700421
542 213.260138905 174.994301597
543 765.407223229 176.079619468
544 115.968217256 175.969323317
545 25.9537151532 175.694569456
546 109.1572145 175.572928953
547 42.5104612004 175.330114231
548 179.870570033 175.338384642
549 66.0586113043 175.139694145
550 127.904686819 175.053968179
551 121.808689295 174.957509341
552 328.498764746 175.235160797
553 80.399267148 175.063976874
554 74.6890941859 174.883121229
555 145.129287586 174.82960714
556 91.3349699924 174.679706535
557 77.3463897794 174.505274067
558 196.74410

788 115.524082308 170.617561545
789 71.8607291514 170.492552896
790 83.724466306 170.382858729
791 63.3755266456 170.247748461
792 355.590110687 170.48147149
793 182.613694825 170.496751368
794 837.318388691 171.335520723
795 15.3347086965 171.139539804
796 41.256665151 170.976575093
797 434.139223401 171.306352848
798 41.4204095364 171.143792218
799 92.9249920444 171.046018717
800 119.681655998 170.981893421
801 72.9002918653 170.859597159
802 259.082788814 170.969464148
803 37.6387634725 170.803629943
804 72.0751451446 170.680985862
805 81.1615476486 170.569919562
806 114.029403284 170.499856964
807 55.4497853694 170.357468262
808 26.768860139 170.179979253
809 83.588432759 170.073076109
810 238.823290239 170.15784826
811 103.964025143 170.076328773
812 102.873449781 169.993668405
813 46.6399238237 169.842128179
814 131.06306521 169.794546506
815 129.141618173 169.744726741
816 7.20113707157 169.545774979
817 25.7067579438 169.36993266
818 42.6497338353 169.215207142
819 78.109815247

1049 53.1949617886 165.37165793
1050 56.8486435573 165.268401018
1051 231.249392074 165.331120592
1052 13.5126823423 165.186943537
1053 177.228360554 165.198368032
1054 14.395902926 165.055427306
1055 342.602457519 165.223558964
1056 85.6411024172 165.148268087
1057 63.8189681661 165.052493702
1058 146.72748862 165.035189636
1059 101.520907154 164.975270502
1060 333.730341953 165.13432335
1061 75.0142000526 165.049464477
1062 94.6863754888 164.983271542
1063 81.8929714178 164.905179155
1064 127.326530382 164.869894039
1065 158.793303748 164.864193673
1066 100.773071818 164.804127017
1067 163.623702529 164.80302175
1068 91.0924548678 164.734068928
1069 7.20113707157 164.586841889
1070 127.039644906 164.551783815
1071 146.221666289 164.534684825
1072 137.991168201 164.509947158
1073 219.697844204 164.561332537
1074 38.5248726226 164.444089319
1075 79.3914914883 164.365044153
1076 73.2106877027 164.280406868
1077 64.1832305168 164.187552344
1078 94.7832178609 164.123229514
1079 77.0345845

...

8613 113.343544767 200.740761582
8614 9.02804206183 200.718508219
8615 47.5376869597 200.700729572
8616 64.319365916 200.68490256
8617 161.857396732 200.680397164
8618 67.0441154414 200.664892316
8619 1123.61788473 200.771963429
8620 209.15646607 200.772935996
8621 581.309664659 200.817071548
8622 944.867558182 200.90335828
8623 85.691065381 200.889998784
8624 90.3950439055 200.877187775
8625 37.4248522283 200.858238976
8626 836.689434034 200.931941444
8627 531.432808695 200.970247062
8628 347.820973436 200.987265341
8629 93.8585063845 200.974851811
8630 127.965979173 200.966392899
8631 16.0267490316 200.94496801
8632 16.0267490316 200.923548084
8633 261.44132207 200.930557323
8634 15.1036086956 200.909037121
8635 131.783936512 200.901032825
8636 585.980053635 200.945617637
8637 286.210788539 200.955488576
8638 332.587092052 200.970725479
8639 229.76727419 200.974058412
8640 412.479492874 200.998535375
8641 49.802086322 200.981039836
8642 650.646644228 201.033066401
8643 198.102467682 

8866 63.5475805557 202.125720509
8867 113.34268279 202.11570889
8868 132.095751484 202.10781398
8869 157.891636997 202.102829067
8870 148.044060552 202.096735191
8871 39.5367272592 202.078412377
8872 74.3317789296 202.064015146
8873 120.930199394 202.054872277
8874 100.70743262 202.043452847
8875 44.5358288224 202.025707509
8876 10.6794046075 202.00415222
8877 531.386559124 202.041253189
8878 213.776097243 202.04257483
8879 220.201670957 202.044619773
8880 260.2768587 202.051176719
8881 370.625852102 202.070156079
8882 685.348097527 202.124560891
8883 120.661512654 202.115391254
8884 70.3948802815 202.100566211
8885 57.0586184699 202.084243687
8886 1168.49522621 202.192988031
8887 345.234652788 202.209081827
8888 167.165656652 202.205139491
8889 157.151549435 202.200071596
8890 225.93670556 202.202741333
8891 161.948964281 202.198214368
8892 78.4142361224 202.184295108
8893 279.487318929 202.1929867
8894 30.9268186734 202.173732494
8895 662.595862565 202.225488578
8896 46.9274021502 20

9120 50.4329507025 202.297087166
9121 469.975054598 202.326431385
9122 240.20726569 202.33058362
9123 292.600419565 202.340477289
9124 115.226739286 202.330930578
9125 41.8260665383 202.313342931
9126 70.2274719428 202.298870939
9127 40.5741914008 202.281153511
9128 688.412014662 202.334404783
9129 324.374068527 202.347771669
9130 40.8422349122 202.330084062
9131 65.6595802122 202.315117953
9132 618.648480193 202.360703562
9133 377.928268779 202.379924885
9134 407.945907747 202.402428003
9135 79.8800140525 202.389017056
9136 50.0432171842 202.372343552
9137 357.154715827 202.389281873
9138 261.344726352 202.395732846
9139 95.7660575431 202.384066579
9140 1997.87642509 202.580488454
9141 94.2367353377 202.568637245
9142 284.403958588 202.577587844
9143 310.005192352 202.589336269
9144 30.6383372945 202.570533536
9145 1678.49648364 202.731907464
9146 57.9307064212 202.716077006
9147 181.490030835 202.713756713
9148 746.087474769 202.773148309
9149 57.6387631581 202.757286628
9150 1091.97

9374 86.279928422 203.186472652
9375 1003.30661882 203.271809698
9376 80.2588448761 203.258691114
9377 852.689508245 203.327941574
9378 345.606105731 203.343111439
9379 219.626970495 203.344847458
9380 139.215457114 203.338011365
9381 66.7471279826 203.323452541
9382 184.324130004 203.321427675
9383 128.787969151 203.313485064
9384 244.112662001 203.317832339
9385 359.206916619 203.334441021
9386 66.7049352714 203.319885837
9387 187.101177011 203.318158237
9388 37.9658672753 203.300546959
9389 123.351881939 203.292032724
9390 146.600796874 203.285995962
9391 62.7906668528 203.271036919
9392 24.913382592 203.25204856
9393 1221.84645368 203.360478878
9394 168.360862029 203.356753533
9395 66.4737512606 203.342185312
9396 99.682119993 203.331154125
9397 164.668180582 203.327040168
9398 93.4616515999 203.315351117
9399 26.295539687 203.296519222
9400 119.637905086 203.287620316
9401 284.399448469 203.296247398
9402 292.609092033 203.305745733
9403 39.3201909217 203.288307882
9404 56.6292675

9624 42.548250142 204.324691857
9625 61.8701931892 204.309892927
9626 232.844363812 204.312856932
9627 100.474430671 204.302071885
9628 1566.09150857 204.443497728
9629 80.9107207045 204.430669817
9630 83.2959402208 204.418092231
9631 96.9687936472 204.406936781
9632 833.461546774 204.472238827
9633 22.6493327989 204.453365783
9634 87.9690882532 204.441276081
9635 195.68156716 204.440367021
9636 85.0614177196 204.427979457
9637 210.254738045 204.428584018
9638 392.121851622 204.448056294
9639 73.8898630586 204.434512913
9640 137.890132439 204.427610685
9641 461.113772716 204.454232357
9642 74.3829073135 204.440743679
9643 132.865409261 204.433321931
9644 154.817130049 204.428177691
9645 164.087622431 204.423995589
9646 64.3800250113 204.409478748
9647 470.765367544 204.437086116
9648 66.2959740879 204.422769491
9649 241.247425359 204.426585518
9650 45.1566155457 204.410082568
9651 373.335021027 204.427584116
9652 96.0439805444 204.416356145
9653 500.949643961 204.447072251
9654 52.0459

In [11]:
np.mean(ppls)

205.50842274089328

In [12]:
def sample_from_distrib(context):
    n = len(context) + 1
    
    # normalize to construct distribution
    probabilities = get_distribution(context)
    print context, sum(probabilities)
    choice = np.random.choice(len(word_to_id) + 1, p=probabilities)
    
    return choice

In [115]:
tokens = [word_to_id['@@start@@'],]

while True:
    try:
        while len(tokens) < 10:
            try:
                tokens.append(sample_from_distrib((tokens[-2],tokens[-1],)))
            except:
                try:
                    tokens.append(sample_from_distrib((tokens[-1],)))
                except KeyError:
                    tokens.append(sample_from_distrib(tuple()))

            if id_to_word[tokens[-1]] == '@@end@@':
                break
    except KeyError:
        continue
    else:
        break

(7332,) 1.0
(22598,) 1.0
(7999,) 1.0
(318,) 1.0
(4402,) 1.0
(11050,) 1.0
(559,) 1.0
(15944,) 1.0
(11853,) 1.0


In [116]:
for id_ in tokens:
    print id_to_word[id_]

@@start@@
@@number@@
they
had
reached
but
one
side
of
death
